In [61]:
#Import dependencies
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import requests
from bs4 import BeautifulSoup
import re
import config
import json
import os
from urllib.parse import urlparse
import csv

In [62]:
#specify the path to chromedriver.exe (download and save on your computer)
driver = webdriver.Chrome()

#open the webpage
driver.get("https://www.instagram.com/")

In [63]:
#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

#enter username and password
username.clear()
username.send_keys(config.username)
password.clear()
password.send_keys(config.password)

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [64]:
# Wait up to 10 seconds for the search button to be clickable on the web page
search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'svg[aria-label="Search"]')))

# Click the search button once it becomes clickable
search_button.click()

In [65]:
#target the search input field
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']")))
searchbox.clear()

#search for the @handle or keyword
keyword = "@marianasdelima"
searchbox.send_keys(keyword)

In [66]:
# Check if the keyword starts with "@"
if keyword.startswith("@"):
    # Remove the "@" symbol
    keyword = keyword[1:]
    
# Find the first element with the specified XPath that matches the keyword    
first_result = driver.find_element(By.XPATH, f'//span[text()="{keyword}"]')

# Click on the found element (assuming it represents the desired search result)
first_result.click()

In [67]:
# Get the initial page height
initial_height = driver.execute_script("return document.body.scrollHeight")

# Create a list to store htmls
soups = []

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for a moment to allow new content to load (adjust as needed)
    time.sleep(5)
    
    # Parse the HTML
    html = driver.page_source
    
    # Create a BeautifulSoup object from the scraped HTML
    soups.append(BeautifulSoup(html, 'html.parser'))

    # Get the current page height
    current_height = driver.execute_script("return document.body.scrollHeight")

    if current_height == initial_height:
        break  # Exit the loop when you can't scroll further

    initial_height = current_height  # Update the initial height for the next iteration

In [69]:
# List to store the post image URLs
post_urls = []

for soup in soups:
    # Find all image elements that match the specific class in the current soup
    elements = soup.find_all('a', class_='x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz _a6hd')

    # Extract the href attributes and filter URLs that start with "/p/" or "/reel/"
    post_urls.extend([element['href'] for element in elements if element['href'].startswith(("/p/", "/reel/"))])
    
# Convert the list to a set to remove duplicates
unique_post_urls = list(set(post_urls))

print(f"before: {len(post_urls)}, after:{len(unique_post_urls)}")

before: 2215, after:608


In [70]:
json_list = []

# Define the query parameters to add
query_parameters = "__a=1&__d=dis"

# go through all urls
for url in unique_post_urls:
    try:
        # Get the current URL of the page
        current_url = driver.current_url

        # Append the query parameters to the current URL
        modified_url = "https://www.instagram.com/" + url + "?" + query_parameters

        # Get URL
        driver.get(modified_url)

        # Wait for a moment to allow new content to load (adjust as needed)
        time.sleep(1)

        # Find the <pre> tag containing the JSON data
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//pre'))
        )
        pre_tag = driver.find_element_by_xpath('//pre')

        # Extract the JSON data from the <pre> tag
        json_script = pre_tag.text

        # Parse the JSON data
        json_parsed = json.loads(json_script)

        # Add json to the list
        json_list.append(json_parsed)
    except (NoSuchElementException, TimeoutException, json.JSONDecodeError) as e:
        print(f"Error processing URL {url}: {e}")


C:\Users\Victor\AppData\Local\Temp\ipykernel_26092\4057388328.py:25: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  pre_tag = driver.find_element_by_xpath('//pre')


Error processing URL /reel/CPLnvshDkx79FKHSc9UFk3JU_2d-_tN-O1okPI0/: Message: 
Stacktrace:
	GetHandleVerifier [0x00B27ED3+47363]
	(No symbol) [0x00AB2E41]
	(No symbol) [0x009A8CAD]
	(No symbol) [0x009DC050]
	(No symbol) [0x009DC51B]
	(No symbol) [0x00A0D832]
	(No symbol) [0x009F8724]
	(No symbol) [0x00A0BDE2]
	(No symbol) [0x009F84D6]
	(No symbol) [0x009D38EF]
	(No symbol) [0x009D4A9D]
	GetHandleVerifier [0x00E2E3C6+3218934]
	GetHandleVerifier [0x00E760B8+3513064]
	GetHandleVerifier [0x00E6FDCC+3487740]
	GetHandleVerifier [0x00BB22C0+613616]
	(No symbol) [0x00ABCD6C]
	(No symbol) [0x00AB8E98]
	(No symbol) [0x00AB8FBD]
	(No symbol) [0x00AAB527]
	BaseThreadInitThunk [0x7664FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77127C6E+286]
	RtlGetAppContainerNamedObjectPath [0x77127C3E+238]



In [71]:
# Lists to store URLs and corresponding dates
all_urls = []
all_dates = []

# Iterate through each JSON data in the list
for json_data in json_list:
    
    # Extract the list from the 'items' key
    item_list = json_data.get('items', [])
    
    # Iterate through each item in the 'items' list
    for item in item_list:
        
        # Extract the date the item was taken
        date_taken = item.get('taken_at')  # Move this line inside the loop

        # Check if 'carousel_media' is present
        carousel_media = item.get('carousel_media', [])
        
        # Iterate through each media in the 'carousel_media' list
        for media in carousel_media:
            
            # Extract the image URL from the media
            image_url = media.get('image_versions2', {}).get('candidates', [{}])[0].get('url')
            
            if image_url:
                # Add the image URL and corresponding date to the lists
                all_urls.append(image_url)
                all_dates.append(date_taken)
                print(f"carousel image added")
                
            # Extract the video URL from the media
            video_versions = media.get('video_versions', [])
            if video_versions:
                video_url = video_versions[0].get('url')
                if video_url:
                    
                    # Add the video URL and corresponding date to the lists
                    all_urls.append(video_url)
                    all_dates.append(date_taken)
                    print(f"carousel video added")

        # Handle cases of unique image, instead of carousel
        image_url = item.get('image_versions2', {}).get('candidates', [{}])[0].get('url')
        if image_url:
            
            # Add the image URL and corresponding date to the lists
            all_urls.append(image_url)
            all_dates.append(date_taken)
            print(f"single image added")

        # Check if 'video_versions' key exists
        video_versions = item.get('video_versions', [])
        if video_versions:
            video_url = video_versions[0].get('url')
            if video_url:
                all_urls.append(video_url)
                all_dates.append(date_taken)
                print(f"video added")
                
# Print or use all collected URLs as needed
print(len(all_urls))
                



single image added
single image added
single image added
single image added
single image added
single image added
single image added
single image added
single image added
carousel image added
carousel image added
single image added
single image added
video added
single image added
single image added
single image added
single image added
carousel image added
carousel image added
carousel image added
carousel video added
carousel image added
carousel image added
carousel image added
carousel image added
carousel image added
single image added
carousel image added
carousel image added
single image added
single image added
carousel image added
carousel image added
single image added
single image added
video added
single image added
single image added
single image added
carousel image added
carousel image added
carousel image added
carousel image added
carousel image added
single image added
single image added
carousel image added
carousel image added
carousel image added
carousel image add

In [72]:
# Create a directory to store downloaded files
download_dir = keyword
os.makedirs(download_dir, exist_ok=True)

# Create subfolders for images and videos
image_dir = os.path.join(download_dir, "images")
video_dir = os.path.join(download_dir, "videos")
os.makedirs(image_dir, exist_ok=True)
os.makedirs(video_dir, exist_ok=True)

# Initialize counters for images and videos
image_counter = 1
video_counter = 1

# Iterate through URLs in the all_urls list and download media
for index, url in enumerate(all_urls, 0):
    response = requests.get(url, stream=True)

    # Extract file extension from the URL
    url_path = urlparse(url).path
    file_extension = os.path.splitext(url_path)[1]

    # Determine the file name based on the URL
    if file_extension.lower() in {'.jpg', '.jpeg', '.png', '.gif'}:
        file_name = f"{all_dates[index]}-img-{image_counter}.png"
        destination_folder = image_dir
        image_counter += 1
    elif file_extension.lower() in {'.mp4', '.avi', '.mkv', '.mov'}:
        file_name = f"{all_dates[index]}-vid-{video_counter}.mp4"
        destination_folder = video_dir
        video_counter += 1
    else:
        # Default to the main download directory for other file types
        file_name = f"{all_dates[index]}{file_extension}"
        destination_folder = download_dir

    # Save the file to the appropriate folder
    file_path = os.path.join(destination_folder, file_name)
    
    # Write the content of the response to the file
    with open(file_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                file.write(chunk)

    print(f"Downloaded: {file_path}")

# Print a message indicating the number of downloaded files and the download directory
print(f"Downloaded {len(all_urls)} files to {download_dir}")

Downloaded: marianasdelima\images\1515301015-img-1.png
Downloaded: marianasdelima\images\1441103297-img-2.png
Downloaded: marianasdelima\images\1557091789-img-3.png
Downloaded: marianasdelima\images\1460692411-img-4.png
Downloaded: marianasdelima\images\1538883715-img-5.png
Downloaded: marianasdelima\images\1476907954-img-6.png
Downloaded: marianasdelima\images\1556133112-img-7.png
Downloaded: marianasdelima\images\1546262038-img-8.png
Downloaded: marianasdelima\images\1414784860-img-9.png
Downloaded: marianasdelima\images\1676821702-img-10.png
Downloaded: marianasdelima\images\1676821702-img-11.png
Downloaded: marianasdelima\images\1676821702-img-12.png
Downloaded: marianasdelima\images\1505412486-img-13.png
Downloaded: marianasdelima\videos\1505412486-vid-1.mp4
Downloaded: marianasdelima\images\1573045056-img-14.png
Downloaded: marianasdelima\images\1334707110-img-15.png
Downloaded: marianasdelima\images\1602167880-img-16.png
Downloaded: marianasdelima\images\1520029695-img-17.png
Do

Downloaded: marianasdelima\images\1545964949-img-136.png
Downloaded: marianasdelima\images\1473468066-img-137.png
Downloaded: marianasdelima\images\1338667974-img-138.png
Downloaded: marianasdelima\images\1638146275-img-139.png
Downloaded: marianasdelima\images\1507607172-img-140.png
Downloaded: marianasdelima\images\1413336690-img-141.png
Downloaded: marianasdelima\images\1549205811-img-142.png
Downloaded: marianasdelima\images\1502063417-img-143.png
Downloaded: marianasdelima\images\1604803737-img-144.png
Downloaded: marianasdelima\images\1537272018-img-145.png
Downloaded: marianasdelima\images\1643996850-img-146.png
Downloaded: marianasdelima\images\1413763202-img-147.png
Downloaded: marianasdelima\images\1634346416-img-148.png
Downloaded: marianasdelima\images\1634346416-img-149.png
Downloaded: marianasdelima\images\1634346416-img-150.png
Downloaded: marianasdelima\images\1634346416-img-151.png
Downloaded: marianasdelima\images\1634346416-img-152.png
Downloaded: marianasdelima\imag

Downloaded: marianasdelima\videos\1701368762-vid-17.mp4
Downloaded: marianasdelima\images\1701368762-img-276.png
Downloaded: marianasdelima\images\1701368762-img-277.png
Downloaded: marianasdelima\images\1701368762-img-278.png
Downloaded: marianasdelima\images\1701368762-img-279.png
Downloaded: marianasdelima\images\1439764003-img-280.png
Downloaded: marianasdelima\images\1656607411-img-281.png
Downloaded: marianasdelima\images\1656607411-img-282.png
Downloaded: marianasdelima\videos\1656607411-vid-18.mp4
Downloaded: marianasdelima\images\1656607411-img-283.png
Downloaded: marianasdelima\images\1656607411-img-284.png
Downloaded: marianasdelima\videos\1656607411-vid-19.mp4
Downloaded: marianasdelima\images\1656607411-img-285.png
Downloaded: marianasdelima\images\1656607411-img-286.png
Downloaded: marianasdelima\videos\1656607411-vid-20.mp4
Downloaded: marianasdelima\images\1656607411-img-287.png
Downloaded: marianasdelima\images\1656607411-img-288.png
Downloaded: marianasdelima\images\1

Downloaded: marianasdelima\images\1571974226-img-412.png
Downloaded: marianasdelima\images\1421192008-img-413.png
Downloaded: marianasdelima\images\1333063497-img-414.png
Downloaded: marianasdelima\images\1428670801-img-415.png
Downloaded: marianasdelima\images\1465521069-img-416.png
Downloaded: marianasdelima\videos\1465521069-vid-28.mp4
Downloaded: marianasdelima\images\1549242140-img-417.png
Downloaded: marianasdelima\images\1341724574-img-418.png
Downloaded: marianasdelima\images\1522617003-img-419.png
Downloaded: marianasdelima\images\1421633162-img-420.png
Downloaded: marianasdelima\images\1422134403-img-421.png
Downloaded: marianasdelima\images\1446905433-img-422.png
Downloaded: marianasdelima\images\1588879566-img-423.png
Downloaded: marianasdelima\images\1374877692-img-424.png
Downloaded: marianasdelima\images\1572129033-img-425.png
Downloaded: marianasdelima\images\1646143121-img-426.png
Downloaded: marianasdelima\images\1531595063-img-427.png
Downloaded: marianasdelima\image

Downloaded: marianasdelima\images\1619631557-img-550.png
Downloaded: marianasdelima\images\1500253379-img-551.png
Downloaded: marianasdelima\images\1633908880-img-552.png
Downloaded: marianasdelima\images\1509454485-img-553.png
Downloaded: marianasdelima\images\1386715999-img-554.png
Downloaded: marianasdelima\images\1645899231-img-555.png
Downloaded: marianasdelima\images\1645899231-img-556.png
Downloaded: marianasdelima\images\1645899231-img-557.png
Downloaded: marianasdelima\images\1645899231-img-558.png
Downloaded: marianasdelima\images\1645899231-img-559.png
Downloaded: marianasdelima\images\1645899231-img-560.png
Downloaded: marianasdelima\images\1645899231-img-561.png
Downloaded: marianasdelima\images\1645899231-img-562.png
Downloaded: marianasdelima\images\1645899231-img-563.png
Downloaded: marianasdelima\images\1645899231-img-564.png
Downloaded: marianasdelima\images\1536239501-img-565.png
Downloaded: marianasdelima\images\1494786047-img-566.png
Downloaded: marianasdelima\imag

Downloaded: marianasdelima\images\1522032343-img-688.png
Downloaded: marianasdelima\images\1522032343-img-689.png
Downloaded: marianasdelima\images\1522032343-img-690.png
Downloaded: marianasdelima\images\1522032343-img-691.png
Downloaded: marianasdelima\images\1522032343-img-692.png
Downloaded: marianasdelima\images\1522032343-img-693.png
Downloaded: marianasdelima\images\1522032343-img-694.png
Downloaded: marianasdelima\images\1520828084-img-695.png
Downloaded: marianasdelima\images\1517790361-img-696.png
Downloaded: marianasdelima\images\1549757110-img-697.png
Downloaded: marianasdelima\images\1584311260-img-698.png
Downloaded: marianasdelima\images\1526412125-img-699.png
Downloaded: marianasdelima\images\1643815588-img-700.png
Downloaded: marianasdelima\images\1643815588-img-701.png
Downloaded: marianasdelima\images\1643815588-img-702.png
Downloaded: marianasdelima\images\1700179079-img-703.png
Downloaded: marianasdelima\images\1700179079-img-704.png
Downloaded: marianasdelima\imag

Downloaded: marianasdelima\images\1645283484-img-822.png
Downloaded: marianasdelima\images\1645283484-img-823.png
Downloaded: marianasdelima\images\1645283484-img-824.png
Downloaded: marianasdelima\images\1645283484-img-825.png
Downloaded: marianasdelima\images\1645283484-img-826.png
Downloaded: marianasdelima\images\1645283484-img-827.png
Downloaded: marianasdelima\images\1502063272-img-828.png
Downloaded: marianasdelima\images\1655434491-img-829.png
Downloaded: marianasdelima\images\1655434491-img-830.png
Downloaded: marianasdelima\images\1655434491-img-831.png
Downloaded: marianasdelima\images\1655434491-img-832.png
Downloaded: marianasdelima\images\1655434491-img-833.png
Downloaded: marianasdelima\images\1655434491-img-834.png
Downloaded: marianasdelima\images\1655434491-img-835.png
Downloaded: marianasdelima\images\1655434491-img-836.png
Downloaded: marianasdelima\images\1655434491-img-837.png
Downloaded: marianasdelima\images\1655434491-img-838.png
Downloaded: marianasdelima\imag

Downloaded: marianasdelima\images\1655490649-img-955.png
Downloaded: marianasdelima\images\1655490649-img-956.png
Downloaded: marianasdelima\images\1655490649-img-957.png
Downloaded: marianasdelima\images\1655490649-img-958.png
Downloaded: marianasdelima\videos\1655490649-vid-63.mp4
Downloaded: marianasdelima\images\1655490649-img-959.png
Downloaded: marianasdelima\images\1438564117-img-960.png
Downloaded: marianasdelima\images\1563235340-img-961.png
Downloaded: marianasdelima\images\1546224348-img-962.png
Downloaded: marianasdelima\images\1491436819-img-963.png
Downloaded: marianasdelima\images\1407359780-img-964.png
Downloaded: marianasdelima\images\1606948229-img-965.png
Downloaded: marianasdelima\images\1606948229-img-966.png
Downloaded: marianasdelima\images\1606948229-img-967.png
Downloaded: marianasdelima\images\1628021271-img-968.png
Downloaded: marianasdelima\images\1523028818-img-969.png
Downloaded: marianasdelima\images\1464280800-img-970.png
Downloaded: marianasdelima\image

Downloaded: marianasdelima\images\1476917274-img-1087.png
Downloaded: marianasdelima\images\1497812309-img-1088.png
Downloaded: marianasdelima\videos\1497812309-vid-74.mp4
Downloaded: marianasdelima\images\1426455828-img-1089.png
Downloaded: marianasdelima\images\1345399838-img-1090.png
Downloaded: marianasdelima\images\1506203206-img-1091.png
Downloaded: marianasdelima\images\1460512398-img-1092.png
Downloaded: marianasdelima\images\1487889719-img-1093.png
Downloaded: marianasdelima\images\1644424333-img-1094.png
Downloaded: marianasdelima\images\1644424333-img-1095.png
Downloaded: marianasdelima\images\1644424333-img-1096.png
Downloaded: marianasdelima\images\1644424333-img-1097.png
Downloaded: marianasdelima\videos\1644424333-vid-75.mp4
Downloaded: marianasdelima\images\1644424333-img-1098.png
Downloaded: marianasdelima\images\1644424333-img-1099.png
Downloaded: marianasdelima\images\1644424333-img-1100.png
Downloaded: marianasdelima\images\1644424333-img-1101.png
Downloaded: marian